In [1]:
import pandas as pd
import numpy as np
import pickle as pk
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import zlib

# manually specify the GPUs to use
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
import math
import random
import MinkowskiEngine as ME
from torch.utils.data import Dataset, DataLoader
from glob import glob
from glob import iglob
import os.path as osp, h5py, tqdm
import utils
from Core import utils
from Core.trainers import Trainer

# System
import time
# Externals
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import tqdm, psutil
from scipy.special import softmax
from sklearn.metrics import classification_report, confusion_matrix, explained_variance_score

/usr/local/lib64/python3.6/site-packages/MinkowskiEngine-0.5.2-py3.6-linux-x86_64.egg/MinkowskiEngine/__init__.py:42: UserWarning: The environment variable `OMP_NUM_THREADS` not set. MinkowskiEngine will automatically set `OMP_NUM_THREADS=16`. If you want to set `OMP_NUM_THREADS` manually, please export it on the command line before running a python script. e.g. `export OMP_NUM_THREADS=12; python your_program.py`. It is recommended to set it below 24.
  "It is recommended to set it below 24.",
/usr/local/lib64/python3.6/site-packages/MinkowskiEngine-0.5.2-py3.6-linux-x86_64.egg/MinkowskiEngine/__init__.py:224: UserWarning: The MinkowskiEngine was compiled with CPU_ONLY flag. If you want to compile with CUDA support, make sure `torch.cuda.is_available()` is True when you install MinkowskiEngine.
  "If you want to compile with CUDA support, make sure `torch.cuda.is_available()` is True when you install MinkowskiEngine.",


In [2]:
# from PDG to label
def pdg2tos(pdg):
    if pdg==13 or pdg==-13 or pdg==211 or pdg==-211:
        return 0 # track
    if pdg==2212 or pdg==321 or pdg==-321:
        return 0 # heavy track
    if pdg==11 or pdg==-11 or pdg==22:
        return 1 # shower
    return -1

In [3]:
# generate events
graph=0
voxel_size=0.05 # cm
numGraphs = len(list(iglob('/eos/home-a/aabedabu/physics_gnn/physics_processed_data_collection/Leigh_grid_data_40000/1GeV_?/mc_sce_dd/*/*.gz')))
for graph_path in iglob('/eos/home-a/aabedabu/physics_gnn/physics_processed_data_collection/Leigh_grid_data_40000/1GeV_?/mc_sce_dd/*/*.gz'):
#numGraphs = len(list(iglob('../../data_grid_leigh/9*/*.gz')))
#for graph_path in iglob('../../data_grid_leigh/9*/*.gz'):
    if graph % 100 == 0:
      print(str(graph+1) + " out of " + str(numGraphs))
    example = {}
    with open(graph_path[:-3]+'.info', 'rb') as info_file:
        lines = info_file.readlines()
        values = {'X':float(lines[0]),
                  'Y':float(lines[1]),
                  'Z':float(lines[2]),
                  'Energy':float(lines[3]),
                  'Interaction':int(lines[4]),
                  'PDG':int(lines[5]),
                  'NumberOfNodes':int(lines[6]),
                  'NumberOfFeatures':int(lines[7])
                  }
        #print(values)
        
    nnodes = values['NumberOfNodes']
    nfeatures = values['NumberOfFeatures']
    
    # spatial coordinates
    XYZ=np.zeros(shape=(nnodes,3),dtype=float)
        
    # all nodes are usable by deafault
    usable = [True]*nnodes
        
    features = []
    X = []
    Y = []
    Z = []
    labels = []
    # iterate over nodes and save spatial coordinates
    with open(graph_path, 'rb') as graph_file:
        codes = np.fromstring(zlib.decompress(graph_file.read()), dtype=np.float32, sep='')
        for node_id in range(nnodes):
            # label
            label = pdg2tos(codes[node_id*(3+nfeatures+1)+(3+nfeatures)])
            if label == -1:
                continue
                
            # features
            feature = []
            for j in range(3,3+values['NumberOfFeatures']):
                feature.append(codes[node_id*(3+values['NumberOfFeatures']+1)+j])
            
            x = codes[node_id*(3+nfeatures+1)+0]
            y = codes[node_id*(3+nfeatures+1)+1]
            z = codes[node_id*(3+nfeatures+1)+2]
            
            x = int(x/voxel_size)
            y = int(y/voxel_size)
            z = int(z/voxel_size)
            xyz = np.dstack((X,Y,Z)).reshape(len(X),3)
            if [x,y,z] in xyz:
                continue
            
            # coordinates
            X.append(x)
            Y.append(y)
            Z.append(z)
            labels.append(label)
            
            ## Removing charge over distance feature
            #del feature[-3:] #delete last three
            
            # get the charge
            #del feature[0:3] 
            #del feature[1:6]
            
            # get the angle
            #del feature[0:5]
            #del feature[1:4]
            
            # Get only the charge over distance
            del feature[0:6]
            
            features.append(feature)
            
    X = np.array(X)
    Y = np.array(Y)
    Z = np.array(Z)
    coords=np.dstack((X,Y,Z)).reshape(X.shape[0],3)
    features = np.array(features)
    labels = np.array(labels).reshape(X.shape[0],1)
    
    example['x'] = features
    example['c'] = coords
    example['y'] = labels
    
    torch.save(example, 'sparse_data_protodune/event{}.pt'.format(graph))
    graph+=1

1 out of 40101


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


101 out of 40101
201 out of 40101
301 out of 40101
401 out of 40101
501 out of 40101
601 out of 40101
701 out of 40101
801 out of 40101
901 out of 40101
1001 out of 40101
1101 out of 40101
1201 out of 40101
1301 out of 40101
1401 out of 40101
1501 out of 40101
1601 out of 40101
1701 out of 40101
1801 out of 40101
1901 out of 40101
2001 out of 40101
2101 out of 40101
2201 out of 40101
2301 out of 40101
2401 out of 40101
2501 out of 40101
2601 out of 40101
2701 out of 40101
2801 out of 40101
2901 out of 40101
3001 out of 40101
3101 out of 40101
3201 out of 40101
3301 out of 40101
3401 out of 40101
3501 out of 40101
3601 out of 40101
3701 out of 40101
3801 out of 40101
3901 out of 40101
4001 out of 40101
4101 out of 40101
4201 out of 40101
4301 out of 40101
4401 out of 40101
4501 out of 40101
4601 out of 40101
4701 out of 40101
4801 out of 40101
4901 out of 40101
5001 out of 40101
5101 out of 40101
5201 out of 40101
5301 out of 40101
5401 out of 40101
5501 out of 40101
5601 out of 40101
5

In [4]:
# collate function
def collate_sparse_minkowski_saul(batch):
    #coords, feats = ME.utils.sparse_collate([d['c'] for d in batch], [d['x'] for d in batch])
    #coords = ME.utils.batched_coordinates([d['c'] for d in batch], dtype=torch.float32)#([d['c'].int() for d in batch])
    voxel_size=1.0
    coords = [(d['c']/voxel_size).int() for d in batch]
    feats = torch.cat([d['x'] for d in batch])
    y = torch.cat([d['y'] for d in batch])
    ret = { 'f': feats, 'c': coords, 'y': y }
    return ret

In [5]:
# dataset
class SparseEventProtoDUNE(Dataset):
    def __init__(self, root, shuffle=True, **kwargs):
        '''Initialiser for SparseEventProtoDUNE class'''
        
        self.root = root
        self.data_files = self.processed_file_names
        if shuffle:
            random.shuffle(self.data_files) 
        self.total_events = len(self.data_files)
        
    @property
    def raw_dir(self):
        return f'{self.root}/raw'
    
    @property
    def processed_dir(self):
        return f'{self.root}'

    @property
    def raw_file_names(self):
        ret = []
        for subdir in glob(f'{self.raw_dir}/*'):
            ret += glob(f'{subdir}/*.root')
        return ret
    
    @property
    def processed_file_names(self):
        return glob(f'{self.processed_dir}/*.pt')
    
    def __len__(self):
        return len(self.data_files)
    
    def __len__(self):
        return self.total_events
    
    def __getitem__(self, idx):
        data = torch.load(self.data_files[idx])
        c = torch.FloatTensor(data['c'])
        x = torch.FloatTensor(data['x'])
        y = torch.LongTensor(data['y'])

        del data
        return { 'x': x, 'c': c, 'y': y}
    
    def vet_files(self):
        for f in self.data_files:
            _, ext = osp.splitext(f)
            if ext != '.pt':
                print('Extension not recognised! Skipping')
                continue
            try:
                torch.load(f)
            except:
                print(f'File {f} is bad! Removing...')
                os.remove(f)

In [6]:
# generate dataset
dataset=SparseEventProtoDUNE("sparse_data_protodune")

In [7]:
number_images = len(dataset)
first_image = dataset[0]
print('Number of images train:', number_images)
#print('First image train:', first_image)¡

Number of images train: 40101


In [8]:
"""
This module defines a generic trainer for simple models and datasets. 
"""

# Locals
from Core.models import get_model
from Core.trainers.base import base
from Core.loss import get_loss
from Core.activation import get_activation
from Core.optim import get_optim
from Core.scheduler import get_scheduler
from Core.metrics import get_metrics
from Core.utils import *

class Trainer(base):
    """Trainer code for basic classification problems with categorical cross entropy."""

    def __init__(self, train_name="test1", summary_dir="summary",
        empty_cache = None, **kwargs):
        super(Trainer, self).__init__(train_name=train_name, **kwargs)
        self.writer = SummaryWriter(f"{summary_dir}/{train_name}")
        self.empty_cache = empty_cache
        
    def arrange_sparse_minkowski(self, data):
        return ME.SparseTensor(
                           features=data['f'], 
                           coordinates=ME.utils.batched_coordinates(data['c'], dtype=torch.float32)#,
                           #quantization_mode=ME.SparseTensorQuantizationMode.RANDOM_SUBSAMPLE, 
                           #device=self.device
                           )
    
    def arrange_truth(self, data):
        return data['y']

    def build_model(self, optimizer_params=None, scheduler_params=None,
        loss_params=None, metric_params=None, name="NodeConv",
        arrange_data="arrange_sparse_minkowski", arrange_truth="arrange_sparse",
        **model_args):
        """Instantiate our model"""

        self.train_name = name
            
        # Construct the model
        #torch.cuda.set_device(self.device)
        self.model = get_model(name=name, **model_args)
        #self.model = self.model.to(self.device)        
        
        # Construct the loss function
        #self.loss_func = get_loss(**loss_params)
        self.loss_func = nn.CrossEntropyLoss()#get_loss(func="categorical_cross_entropy")

        # Construct the optimizer
        #self.optimizer = get_optim(model_params=self.model.parameters(), **optimizer_params)
        self.optimizer = optim.SGD(self.model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
        #self.scheduler = get_scheduler(self.optimizer, **scheduler_params)
        self.scheduler = optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=3)
        '''
        scheduler_params = {"ReduceLROnPlateau":{"factor":0.9,"patience":0.3},
                            "StepLR":{"step_size":5, "gamma":0.8}}
        self.scheduler = get_scheduler(self.optimizer, "ReduceLROnPlateau")
        '''
        self.scheduler = None
        
        'Classification'
        # Configure metrics
        '''
        metrics=metric_params["metrics"]
        metric_args = metric_params[metrics]
        self.metrics = get_metrics(metrics)(**metric_args)
        '''
        #self.metrics = get_metrics("Classification")(["track", "heavy_track", "shower"])
        self.metrics = get_metrics("Classification")(["track", "shower"])
        #self.metrics = get_metrics("Classification")(["protons", "muons"])
        #self.batch_metrics = metric_params["include_batch_metrics"]
        self.batch_metrics = "yes"
        
        '''
        # Select function to arrange data
        self.arrange_data = get_arrange_data(arrange_data)
        self.arrange_truth = get_arrange_truth(arrange_truth)
        '''

    def load_state_dict(self, state_dict, **kwargs):
        """Load state dict from trained model"""
        self.model.load_state_dict(torch.load(state_dict, map_location=f"cuda:{self.device}")["model"])

    def train_epoch(self, data_loader, **kwargs):
        """Train for one epoch"""
        self.model.train()
        self.metrics.new_epoch()
        summary = dict()
        sum_loss = 0.
        start_time = time.time()
        # Loop over training batches
        batch_size = data_loader.batch_size
        n_batches = int(math.ceil(len(data_loader.dataset)/batch_size)) #if max_iters_train is None else max_iters_train
        t = tqdm.tqdm(enumerate(data_loader),total=n_batches)
        for i, data in t:
            self.optimizer.zero_grad()
            # Different input shapes for SparseConvNet vs MinkowskiEngine
            batch_input = self.arrange_sparse_minkowski(data)
            batch_output = self.model(batch_input)
            batch_target = self.arrange_truth(data)#.to(self.device)
            batch_target = torch.reshape(batch_target, (len(batch_target),))
            
            batch_loss = nn.functional.cross_entropy(batch_output.F, batch_target, reduction="mean")
            batch_loss.backward()

            # Calculate accuracy
            metrics = self.metrics.train_batch_metrics(batch_output.F, batch_target)
            
            self.optimizer.step()

            sum_loss += batch_loss.item()
            t.set_description("loss = %.5f" % batch_loss.item() )
            t.refresh() # to show immediately the update

            # add to tensorboard summary
            if self.batch_metrics:
                metrics = self.metrics.train_batch_metrics(batch_output.F, batch_target)
                if self.iteration%100 == 0:
                    self.writer.add_scalar("loss/batch", batch_loss.item(), self.iteration)
                    for key, val in metrics.items(): self.writer.add_scalar(key, val, self.iteration)
            self.iteration += 1

            if self.empty_cache is not None and self.iteration % self.empty_cache == 0:
                #torch.cuda.empty_cache()
                print("")

        summary["lr"] = self.optimizer.param_groups[0]["lr"]
        summary["train_time"] = time.time() - start_time
        summary["train_loss"] = sum_loss / n_batches
        self.logger.debug(" Processed %i batches", n_batches)
        self.logger.info("  Training loss: %.3f", summary["train_loss"])
        self.logger.info("  Learning rate: %.5f", summary["lr"])
        return summary
    
    @torch.no_grad()
    def evaluate(self, data_loader, max_iters_eval=None, **kwargs):
        """Evaluate the model"""
        self.model.eval()
        summary = dict()
        sum_loss = 0
        start_time = time.time()
        # Loop over batches
        batch_size = data_loader.batch_size
        n_batches = int(math.ceil(len(data_loader.dataset)/batch_size))
        t = tqdm.tqdm(enumerate(data_loader),total=n_batches)
        true=[]
        pred=[]
        for i, data in t:
            batch_input = self.arrange_sparse_minkowski(data)
            batch_output = self.model(batch_input)
            batch_target = self.arrange_truth(data).to(batch_output.device)
            batch_target = torch.reshape(batch_target, (len(batch_target),))
            
            
            pred+=np.argmax(softmax(batch_output.F.cpu().detach().numpy(),axis=-1), axis=-1).tolist()
            true+=batch_target.cpu().detach().numpy().tolist()
            
            batch_loss = self.loss_func(batch_output.F, batch_target)
            sum_loss += batch_loss.item()
            self.metrics.valid_batch_metrics(batch_output.F, batch_target)
        summary["valid_time"] = time.time() - start_time
        summary["valid_loss"] = sum_loss / n_batches
        self.logger.debug(" Processed %i samples in %i batches",
                          len(data_loader.sampler), n_batches)
        self.logger.info("  Validation loss: %.3f" % (summary["valid_loss"]))
        return summary, np.array(true), np.array(pred)
    
    def train(self, train_data_loader, n_epochs, resume=False, valid_data_loader=None, sherpa_study=None, sherpa_trial=None, **kwargs):
        """Run the model training"""

        # Loop over epochs
        best_valid_loss = 99999
        self.first_epoch = 0
        if resume:
            self.logger.info("Resuming existing training!")
            state_dict = None
            while True:
                state_files = glob(f"{self.output_dir}/checkpoints/*{self.first_epoch:03d}.pth.tar")
                if len(state_files) > 1:
                    raise Exception(f"More than one state file found for epoch {self.first_epoch}!")
                elif len(state_files) == 0:
                    self.logger.info(f"Resuming training from epoch {self.first_epoch}.")
                    self.load_state_dict(state_dict)
                    break
                state_dict = state_files[0]
                self.first_epoch += 1
        n_batches = int(math.ceil(len(train_data_loader.dataset)/train_data_loader.batch_size))
        self.iteration = self.first_epoch * n_batches
        for i in range(self.first_epoch, n_epochs):
            self.logger.info("Epoch %i" % i)
            self.writer.add_scalar("learning_rate", self.optimizer.param_groups[0]["lr"], i+1)
            summary = dict(epoch=i)
            # Train on this epoch
            sum_train = self.train_epoch(train_data_loader, **kwargs)
            summary.update(sum_train)
            # Evaluate on this epoch
            sum_valid = None
            if valid_data_loader is not None:
                sum_valid, true_pid, pred_pid = self.evaluate(valid_data_loader, **kwargs)
                summary.update(sum_valid)

                if sum_valid["valid_loss"] < best_valid_loss:
                    best_valid_loss = sum_valid["valid_loss"]
                    self.logger.debug("Checkpointing new best model with loss: %.3f", best_valid_loss)
                    self.write_checkpoint(checkpoint_id=i,best=True)

            if self.scheduler is not None:
                self.scheduler.step(sum_valid["valid_loss"])

            # Save summary, checkpoint
            self.save_summary(summary)
            if self.output_dir is not None:
                self.write_checkpoint(checkpoint_id=i)

            self.writer.add_scalars('loss/epoch', {
                'train': summary['train_loss'],
                'valid': summary['valid_loss'] }, i+1)
            metrics = self.metrics.epoch_metrics()
            if sherpa_study is not None and sherpa_trial is not None:
                sherpa_study.add_observation(
                trial=sherpa_trial,
                iteration=i,
                objective=metrics["acc/epoch"]["valid"])
            for key, val in metrics.items(): self.writer.add_scalars(key, val, i+1)
            for key, val in metrics.items(): print(key, val)
            if valid_data_loader is not None:
                print(confusion_matrix(pred_pid, true_pid))
            

        return self.summaries
    
def _test():
    t = Trainer(output_dir="./")
    t.build_model()


In [9]:
fulllen = len(dataset)
t_v_split= 0.2
tv_num = math.ceil(fulllen*t_v_split)
#splits = np.cumsum([fulllen-tv_num,0,tv_num])
splits=np.cumsum([fulllen-2*tv_num,0,tv_num, 0, tv_num])
print(splits)
collate = collate_sparse_minkowski_saul

train_dataset = torch.utils.data.Subset(dataset,np.arange(start=0,stop=splits[0]))
valid_dataset = torch.utils.data.Subset(dataset,np.arange(start=splits[1],stop=splits[2]))
test_dataset = torch.utils.data.Subset(dataset,np.arange(start=splits[3],stop=splits[4]))
train_loader = DataLoader(train_dataset, collate_fn=collate, batch_size=32, num_workers=0, shuffle=True)#, pin_memory=True)
valid_loader = DataLoader(valid_dataset, collate_fn=collate, batch_size=32, num_workers=0, shuffle=False)
test_loader = DataLoader(test_dataset, collate_fn=collate, batch_size=32, num_workers=0, shuffle=False)

[24059 24059 32080 32080 40101]


In [10]:
trainer = Trainer(device=0, n_epochs=10, max_iters_train=None, max_iters_eval=None,
                  summary_dir='summary/3d/test', output_dir='./')
name = "MinkUNet14A"
#name = "MinkUNet14B"
#name = "MinkUNet14C"
#name = "MinkUNet14D"
#name = "MinkUNet18A"
#name = "MinkUNet18B"
#name = "MinkUNet18C"
#name = "MinkUNet18D"
#name = "MinkUNet34A"
#name = "MinkUNet34B"
#name = "MinkUNet34C"
#name = "MinkUNet34D"
trainer.build_model(name=name,
                    in_channels=3, out_channels=2, D=3
                   )

In [11]:
# train
trainer.train(train_data_loader=train_loader, valid_data_loader=valid_loader, n_epochs=30)

  0%|          | 0/752 [00:00<?, ?it/s]/usr/local/lib64/python3.6/site-packages/MinkowskiEngine-0.5.2-py3.6-linux-x86_64.egg/MinkowskiEngine/MinkowskiSparseTensor.py:288: UserWarning: coordinates implicitly converted to torch.IntTensor. To remove this warning, use `.int()` to convert the coords into an torch.IntTensor
  + "coords into an torch.IntTensor"
/usr/local/lib64/python3.6/site-packages/torch/autograd/__init__.py:132: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  allow_unreachable=True)  # allow_unreachable flag
100%|██████████| 251/251 [06:14<00:00,  1.49s/it]


acc/epoch {'train': 91.11030019729618, 'valid': 93.20492389207304}
acc_class/epoch/track {'train': 93.33102688498148, 'valid': 95.53967593843376}
acc_class/epoch/shower {'train': 87.41603771001981, 'valid': 89.17765540593611}
time/epoch {'train': 1497.5179750919342, 'valid': 374.40743613243103}


  0%|          | 0/752 [00:00<?, ?it/s]

[[944189  62005]
 [ 44080 510930]]


/usr/local/lib64/python3.6/site-packages/MinkowskiEngine-0.5.2-py3.6-linux-x86_64.egg/MinkowskiEngine/MinkowskiSparseTensor.py:288: UserWarning: coordinates implicitly converted to torch.IntTensor. To remove this warning, use `.int()` to convert the coords into an torch.IntTensor
  + "coords into an torch.IntTensor"
100%|██████████| 251/251 [05:51<00:00,  1.40s/it]


...

  0%|          | 0/752 [00:00<?, ?it/s]

[[950798  52356]
 [ 37471 520579]]


/usr/local/lib64/python3.6/site-packages/MinkowskiEngine-0.5.2-py3.6-linux-x86_64.egg/MinkowskiEngine/MinkowskiSparseTensor.py:288: UserWarning: coordinates implicitly converted to torch.IntTensor. To remove this warning, use `.int()` to convert the coords into an torch.IntTensor
  + "coords into an torch.IntTensor"
100%|██████████| 251/251 [01:30<00:00,  2.78it/s]


acc/epoch {'train': 96.47550158695691, 'valid': 93.88676944204602}
acc_class/epoch/track {'train': 96.56062604643093, 'valid': 95.02979451950836}
acc_class/epoch/shower {'train': 96.3338938424612, 'valid': 91.9151387155611}
time/epoch {'train': 644.6622936725616, 'valid': 90.16893649101257}
[[939150  46321]
 [ 49119 526614]]


{'epoch': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29],
 'lr': [0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1,
  0.1],
 'train_time': [1497.5195684432983,
  1454.53582239151,
  1304.795478105545,
  1154.4667110443115,
  1102.3687596321106,
  1103.8568894863129,
  1112.2233304977417,
  867.8346235752106,
  621.8738601207733,
  620.86230301857,
  608.9378273487091,
  610.4895186424255,
  618.4660513401031,
  616.4182307720184,
  619.3359639644623,
  637.3534896373749,
  621.3266496658325,
  617.5429074764252,
  614.4940659999847,
  612.8371181488037,
  615.848997592926,
  685.7720775604248,
  871.405766248703,
  1146.4812502861023,
  1125.858561515808,
  715.8278558254242,
  642.0602223873138,
  638.4583523273468,


In [12]:
# test
summary, true_pid, pred_pid = trainer.evaluate(data_loader=test_loader)

  0%|          | 0/251 [00:00<?, ?it/s]/usr/local/lib64/python3.6/site-packages/MinkowskiEngine-0.5.2-py3.6-linux-x86_64.egg/MinkowskiEngine/MinkowskiSparseTensor.py:288: UserWarning: coordinates implicitly converted to torch.IntTensor. To remove this warning, use `.int()` to convert the coords into an torch.IntTensor
  + "coords into an torch.IntTensor"
100%|██████████| 251/251 [04:18<00:00,  1.03s/it]


In [13]:
# results on test
#target_names = ["track","heavy_track", "shower"]
target_names = ["track", "shower"]
print(classification_report(true_pid, pred_pid, digits=3, target_names=target_names))
print(confusion_matrix(pred_pid, true_pid))

              precision    recall  f1-score   support

       track      0.956     0.953     0.955   1046808
      shower      0.915     0.921     0.918    577258

    accuracy                          0.941   1624066
   macro avg      0.936     0.937     0.936   1624066
weighted avg      0.942     0.941     0.942   1624066

[[997503  45707]
 [ 49305 531551]]
